In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/swish'

In [4]:
# !pip install smt

In [5]:
def true_1D_2(x): #True function for 1D_1 dy/dx = cos(0.01*x) BC1: y(0)=0; x \in [-100,100]
    y = 100*np.sin(0.01*x)
    return y
    

In [6]:
label = "1D_FODE_swish_"
loss_thresh = 0.005

x = np.linspace(-600,600,5000).reshape(-1,1)
ysol = true_1D_2(x)

bc1_x = np.array(0).reshape(-1,1) 
bc1_y = np.array(0).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)

 
x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)

y_true = true_1D_2(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

#torch.autograd.set_detect_anomaly(True)

In [7]:
def colloc_pts(N_f,seed):

  #Collocation Points
  # Latin Hypercube sampling for collocation points 
  # N_f sets of tuples(x,y)
  x01 = np.array([[0.0, 1.0]])
  sampling = LHS(xlimits=x01,random_state =seed)

  x_coll_train = lb + (ub-lb)*sampling(N_f)
  x_coll_train = np.vstack((x_coll_train, bc1_x)) # append training points to collocation points 

  return x_coll_train

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Sigmoid()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.beta = Parameter(torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
        
              
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = z*self.activation(self.beta[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        f = dy_dx - torch.cos(0.01*g)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + 100*loss_f
        
        return loss_val
     
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred

    def test_loss(self):
        y_pred = self.test()
        
       
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
      
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re 

In [9]:
def train_step(seed):
    x_coll_np_array = colloc_pts(N_f,seed*123)
    x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_coll_train,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        
        return loss

    optimizer.step(closure)

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_model(max_iter,rep): 
  print(rep) 
  torch.manual_seed(rep*11)
  start_time = time.time() 
  thresh_flag = 0
  x_coll = torch.from_numpy(colloc_pts(N_f,123)).float().to(device)
  f_hat = torch.zeros(x_coll.shape[0],1).to(device)

  for i in range(max_iter):
    
    train_step(i)

    loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_coll,f_hat).cpu().detach().numpy()
    if(thresh_flag == 0):
        if(loss_np < loss_thresh):
            time_threshold[rep] = time.time() - start_time
            epoch_threshold[rep] = i+1            
            thresh_flag = 1       
    data_update(loss_np)
    print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

  elapsed_time[rep] = time.time() - start_time  
  print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:
 
max_reps = 10
max_iter = 100

train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))


for reps in range(max_reps):   
  train_loss = []
  test_mse_loss = []
  test_re_loss = []   
  beta_val = []

  torch.manual_seed(reps*36)
  N_f = 10000 #Total number of collocation points

  layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
  PINN = Sequentialmodel(layers)

  PINN.to(device)

  'Neural Network Summary'
  print(PINN)

  params = list(PINN.parameters())

  optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                            max_iter = 10, 
                            max_eval = 15, 
                            tolerance_grad = 1e-6, 
                            tolerance_change = 1e-6, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')



  train_model(max_iter,reps)


  torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
  train_loss_full.append(train_loss)
  test_mse_full.append(test_mse_loss)
  test_re_full.append(test_re_loss)
  beta_full.append(beta_val)


  print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 47.7695 Test MSE 5222.675787360136 Test RE 0.9999993045627861
1 Train Loss 47.7695 Test MSE 5222.675787360136 Test RE 0.9999993045627861
2 Train Loss 47.7695 Test MSE 5222.675787360136 Test RE 0.9999993045627861
3 Train Loss 47.7695 Test MSE 5222.675787360136 Test RE 0.99999930456278

1 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
2 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
3 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
4 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
5 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
6 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
7 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
8 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
9 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
10 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
11 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
12 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
13 Train Loss 47.7695 Test MSE 5222.677250724914 Test RE 0.9999994446599045
14 Train Loss 47.7695

6 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
7 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
8 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
9 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
10 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
11 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
12 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
13 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
14 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
15 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
16 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
17 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003689423973
18 Train Loss 47.769505 Test MSE 5222.686905193035 Test RE 1.0000003

6 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
7 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
8 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
9 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
10 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
11 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
12 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
13 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
14 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
15 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
16 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
17 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
18 Train Loss 47.7695 Test MSE 5222.676831227249 Test RE 0.9999994044987551
19 Train Loss 47

10 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
11 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
12 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
13 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
14 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
15 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
16 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
17 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
18 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
19 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
20 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
21 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852
22 Train Loss 47.769505 Test MSE 5222.691690389984 Test RE 1.000000827058852

8 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
9 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
10 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
11 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
12 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
13 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
14 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
15 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
16 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
17 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
18 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
19 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
20 Train Loss 47.7695 Test MSE 5222.682019008245 Test RE 0.9999999011575363
21 Train Loss 

6 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
7 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
8 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
9 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
10 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
11 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
12 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
13 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
14 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
15 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
16 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
17 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013894288033
18 Train Loss 47.769512 Test MSE 5222.697564556521 Test RE 1.0000013

4 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
5 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
6 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
7 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
8 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
9 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
10 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
11 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
12 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
13 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
14 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
15 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.9999995123636262
16 Train Loss 47.769497 Test MSE 5222.677957914704 Test RE 0.999999512

4 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
5 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
6 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
7 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
8 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
9 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
10 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
11 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
12 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
13 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
14 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
15 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.9999987948364473
16 Train Loss 47.769497 Test MSE 5222.670463086977 Test RE 0.999998794

2 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
3 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
4 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
5 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
6 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
7 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
8 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
9 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
10 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
11 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
12 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
13 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
14 Train Loss 47.7695 Test MSE 5222.685989605434 Test RE 1.0000002812875182
15 Train Loss 47.769

In [13]:
label

'1D_FODE_swish_'

In [14]:
import scipy.io as sio

In [15]:
for tune_reps in range(4,5):
    label = "1D_FODE_swish_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(i,' ',np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: '1D_FODE_swish_tune4.mat'